<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# =========================
# Google Colab: CPU version of text-generation-webui
# =========================

import os
from pathlib import Path

# Force CPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Go to /content so we always have a valid path
%cd /content

# Clone the web UI if not present
if not Path("/content/text-generation-webui").exists():
    print("Installing text-generation-webui...")
    !git clone https://github.com/oobabooga/text-generation-webui
else:
    print("text-generation-webui already exists.")

%cd /content/text-generation-webui

# Install CPU-friendly dependencies
!./setup.sh || echo "setup.sh failed, installing minimal deps"
!pip install --upgrade gradio huggingface_hub torch

# =========================
# Parameters (change these if needed)
# =========================
model_url = "https://huggingface.co/turboderp/gemma-2-9b-it-exl2"  # your HF model
branch = "main"
command_line_flags = "--cpu --load-in-8bit --no-half --no-flash-attn"
api = False

# Add API flags if requested
if api:
    for param in ['--api', '--public-api']:
        if param not in command_line_flags:
            command_line_flags += f" {param}"

# Download the model
if model_url != "":
    if not model_url.startswith("http"):
        model_url = "https://huggingface.co/" + model_url
    url_parts = model_url.strip("/").split("/")
    output_folder = f"{url_parts[-2]}_{url_parts[-1]}"
    if branch not in ['', 'main']:
        output_folder += f"_{branch}"
        !python download-model.py {model_url} --branch {branch}
    else:
        !python download-model.py {model_url}
else:
    output_folder = ""

# =========================
# Launch the web UI
# =========================
cmd = f"./start_linux.sh {command_line_flags} --share"
if output_folder != "":
    cmd += f" --model {output_folder}"

print("Launching the web UI on CPU...")
!$cmd
